In [8]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=""
import re
import numpy as np
import random
np.random.seed(1337) 
from Bio import SeqIO

import keras
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import RandomUniform

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("input_file", help="Enter the path to the input fasta file",type=str)
args = parser.parse_args()
seqs =  args.input_file



usage: ipykernel_launcher.py [-h] input_file
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/hilal/anaconda3/envs/tf9/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
def get_m6a_model():
    inp = Input((41,4))
    x= Conv1D(8,5,padding='same',activation='relu',kernel_initializer=keras.initializers.RandomUniform())(inp)
    x= Conv1D(4,3,padding='same',activation='relu',kernel_initializer=keras.initializers.RandomUniform())(x)
    x = Flatten()(x)
    x = Dropout(0.25)(x)
    prediction = Dense(1,activation='sigmoid')(x)
    model = Model(inp,prediction)
    opt = Adam(lr=0.0001,amsgrad = True)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.load_weights("best_weights.hdf5")
    return model

In [3]:
model = get_m6a_model()

In [4]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 41, 4)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 41, 8)             168       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 41, 4)             100       
_________________________________________________________________
flatten_1 (Flatten)          (None, 164)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 164)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 165       
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
def check(line):
    if (re.match('[ACTGactg]',line))and(len(re.findall('\S',line))==41):
        return True 
    else:
        return False

    
def get_onehot(seq): 
    seq = str(seq).upper()
    d = np.array(['A','C','G','T'])
    y = np.frombuffer(seq, dtype='|S1')[:, np.newaxis] == d
    return y


In [6]:
def process_fasta(inpfile,outfile,model,thresh):
    res= open("results/"+outfile+'.txt','w')
    cnt = 0
    print inpfile
    output = []
    
    for record in SeqIO.parse('UPLOAD_FOLDER/'+inpfile, "fasta"):
        seq=str(record.seq)
        if check(seq)==False:
            continue
     
        idx = str(record.id)
        one_hot = np.expand_dims(np.asarray(get_onehot(seq),dtype=np.float), axis=0)
        pred= model.predict(one_hot)

        if pred[0][0]>=thresh:
            tmp = [idx,seq, 'i6mA site']
            res.write(str(record.id)+','+'i6mA site'+'\n')
        else:
            tmp = [idx, seq, 'Not a '+'i6mA site']
            res.write(str(record.id)+','+'Not an '+ 'i6mA site'+'\n')
        output.append(tmp)
        cnt+=1
        if cnt == 1000:
            break
    res.close()
    return output        
        

In [7]:
process_fasta('test.fa','test.txt', model,0.5)

test.fa


[['seq1', 'AATTGGATAGGGAGAAGCCGATGTAGCTGATTCTAGCAAGA', 'i6mA site'],
 ['seq2', 'GTATATAACTTTTTTCTTCAAGGCAGCAGGTGTCTGCCTAA', 'i6mA site'],
 ['seq3', 'AACGGGTGGACGTCCACCCGAATGATTAGAATCCCTCTCCA', 'i6mA site'],
 ['seq4', 'GAGCAATTTAGGGATGAGTGACCGACCGGGAAATTCTTCTC', 'i6mA site'],
 ['seq5', 'CCCAGGCCGGGCCCGCTTAAATCTGGCAGCTCTCATAGGTC', 'i6mA site'],
 ['seq6', 'AGGGACATAATCACGTTTCGAGGCAAATTTTGAATATATTT', 'i6mA site'],
 ['seq7', 'AAAAAAAATGATGGAAATTGAGGTACCACCAGTGTCAGTAT', 'i6mA site'],
 ['seq8', 'GCAAAGGGGTTGAGAAAAAGATGTACCAAGAAATCCAAGGG', 'i6mA site'],
 ['seq9', 'AATACATGGGGTTATGTGCCACCGGTCATAATATCTAGGGT', 'i6mA site'],
 ['seq10', 'CATAAATATATGGTTTACTGATATGGCAGCAAATTCCGAGA', 'i6mA site'],
 ['seq11', 'GGCAACGAAATCATATTCCCAGTTGAAACTCAAGAAGGGCC', 'i6mA site'],
 ['seq12', 'GAAAGTCTTATGACTTGTGTATTGTCTACCTTGCCAGCATC', 'Not a i6mA site'],
 ['seq13', 'GTATTGTCTACCTTGCCAGCATCTTGTGAGCAGCATAGTGT', 'Not a i6mA site'],
 ['seq14', 'GCAGCATAGTGTTGCTACCAATTTGGATAATCCTGGAAACA', 'Not a i6mA site'],
 ['seq15', 